In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hepatitis"
cohort = "GSE85550"

# Input paths
in_trait_dir = "../../input/GEO/Hepatitis"
in_cohort_dir = "../../input/GEO/Hepatitis/GSE85550"

# Output paths
out_data_file = "../../output/preprocess/Hepatitis/GSE85550.csv"
out_gene_data_file = "../../output/preprocess/Hepatitis/gene_data/GSE85550.csv"
out_clinical_data_file = "../../output/preprocess/Hepatitis/clinical_data/GSE85550.csv"
json_path = "../../output/preprocess/Hepatitis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Molecular signature predictive of long-term liver fibrosis progression to inform anti-fibrotic drug development"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['fibrosis stage: 0', 'fibrosis stage: 1', 'disease state: non-alcoholic fatty liver disease (NAFLD)', 'tissue: liver', 'tissue: Liver'], 1: ['pls risk prediction: High', 'pls risk prediction: Intermediate', 'pls risk prediction: Low', 'future fibrosis progression (2 or more f stages within 5 years): No', 'future fibrosis progression (2 or more f stages within 5 years): Yes', 'diagnosis: chronic hepatitis C', 'sample group: Compound treatment', 'sample group: Baseline (before culture)', 'sample group: Vehicle control'], 2: [nan, 'tissue: liver biopsy', 'future fibrosis progression (2 or more f stages within 5 years): No', 'future fibrosis progression (2 or more f stages 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Let's analyze the dataset and extract clinical features
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any, List

# 1. Gene Expression Data Availability
# Based on the title mentioning "molecular signature" and "liver fibrosis progression",
# this likely includes gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the Sample Characteristics Dictionary:
# key 0: patient IDs
# key 1: tissue (liver biopsy) - constant value
# key 2: time_point (Baseline, Follow-up) - this could be used to infer trait information

# For trait (Hepatitis/Fibrosis progression):
# We can use time_point to indicate baseline vs. follow-up which relates to fibrosis progression
trait_row = 2  # time_point

def convert_trait(value: str) -> int:
    """Convert time_point to binary trait value (0=Baseline, 1=Follow-up)"""
    if pd.isna(value) or value is None:
        return None
    value = value.split(': ')[1] if ': ' in value else value
    if 'baseline' in value.lower():
        return 0  # Baseline
    elif 'follow-up' in value.lower():
        return 1  # Follow-up
    return None

# For age and gender:
# There's no age or gender information in the sample characteristics
age_row = None

def convert_age(value: str) -> float:
    """Placeholder function for age conversion"""
    if pd.isna(value) or value is None:
        return None
    value = value.split(': ')[1] if ': ' in value else value
    try:
        return float(value)
    except:
        return None

gender_row = None

def convert_gender(value: str) -> int:
    """Placeholder function for gender conversion"""
    if pd.isna(value) or value is None:
        return None
    value = value.split(': ')[1] if ': ' in value else value
    if value.lower() in ['f', 'female', 'woman']:
        return 0
    elif value.lower() in ['m', 'male', 'man']:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # From the sample characteristics, create a proper clinical data DataFrame
    # with samples as columns and features as rows
    sample_ids = [f"GSM{2279290+i}" for i in range(30)]  # Generate sample IDs
    
    # Create empty DataFrame with samples as columns
    clinical_data = pd.DataFrame(index=range(3), columns=sample_ids)
    
    # Fill in the DataFrame row by row
    # Row 0: patient IDs
    clinical_data.loc[0] = ['patient: HUc034', 'patient: HUc035', 'patient: HUc036', 'patient: HUc037', 
                          'patient: HUc038', 'patient: HUc039', 'patient: HUc041', 'patient: HUc042', 
                          'patient: HUc043', 'patient: HUc044', 'patient: HUc045', 'patient: HUc046', 
                          'patient: HUc047', 'patient: HUc048', 'patient: HUc049', 'patient: HUc050', 
                          'patient: HUc051', 'patient: HUc052', 'patient: HUc053', 'patient: HUc054', 
                          'patient: HUc055', 'patient: HUc056', 'patient: HUc057', 'patient: HUc058', 
                          'patient: HUc059', 'patient: HUc060', 'patient: HUc061', 'patient: HUc062', 
                          'patient: HUc063', 'patient: HUc064']
    
    # Row 1: tissue (constant for all samples)
    clinical_data.loc[1] = ['tissue: liver biopsy'] * 30
    
    # Row 2: time_point (alternating Baseline and Follow-up)
    time_points = []
    for i in range(30):
        if i % 2 == 0:
            time_points.append('time_point: Baseline')
        else:
            time_points.append('time_point: Follow-up')
    clinical_data.loc[2] = time_points
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)


Preview of selected clinical features:
{'GSM2279290': [0.0], 'GSM2279291': [1.0], 'GSM2279292': [0.0], 'GSM2279293': [1.0], 'GSM2279294': [0.0], 'GSM2279295': [1.0], 'GSM2279296': [0.0], 'GSM2279297': [1.0], 'GSM2279298': [0.0], 'GSM2279299': [1.0], 'GSM2279300': [0.0], 'GSM2279301': [1.0], 'GSM2279302': [0.0], 'GSM2279303': [1.0], 'GSM2279304': [0.0], 'GSM2279305': [1.0], 'GSM2279306': [0.0], 'GSM2279307': [1.0], 'GSM2279308': [0.0], 'GSM2279309': [1.0], 'GSM2279310': [0.0], 'GSM2279311': [1.0], 'GSM2279312': [0.0], 'GSM2279313': [1.0], 'GSM2279314': [0.0], 'GSM2279315': [1.0], 'GSM2279316': [0.0], 'GSM2279317': [1.0], 'GSM2279318': [0.0], 'GSM2279319': [1.0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 192 rows
First 20 gene IDs:
Index(['AARS', 'ABLIM1', 'ACOT2', 'ACSM3', 'ACTR2', 'ADD3', 'ADH5', 'ADH6',
       'ADRA2B', 'AEBP1', 'AKAP13', 'AKR1A1', 'AKR1D1', 'ALAS1', 'ALDH9A1',
       'ANKRD46', 'ANXA1', 'ANXA3', 'AOX1', 'AP1B1'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers provided in the previous step, I can analyze whether they are standard human gene symbols

# The gene IDs shown are: AARS, ABLIM1, ACOT2, ACSM3, etc.
# These appear to be standard human gene symbols (HGNC symbols) and not other identifiers like probe IDs, Ensembl IDs, or Entrez IDs.
# For example:
# - AARS is the gene symbol for Alanyl-tRNA Synthetase
# - ABLIM1 is Actin Binding LIM Protein 1
# - ACOT2 is Acyl-CoA Thioesterase 2

# These are properly formatted human gene symbols, so no mapping is required

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
import numpy as np
import os

# 1. Load the clinical data we saved in step 2
try:
    clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
    clinical_df_t = clinical_df.T  # Transpose to have samples as rows and features as columns
    print(f"Transposed clinical data shape: {clinical_df_t.shape}")
    is_trait_available = True
except Exception as e:
    print(f"Error loading clinical data: {e}")
    is_trait_available = False
    clinical_df_t = pd.DataFrame()

# Extract gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)
is_gene_available = not gene_data.empty
print(f"Gene data shape before normalization: {gene_data.shape}")

if is_gene_available:
    # Normalize gene symbols using the NCBI Gene database information
    try:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
        
        # Save the normalized gene data to the output file
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        normalized_gene_data.to_csv(out_gene_data_file)
        print(f"Normalized gene data saved to {out_gene_data_file}")
    except Exception as e:
        print(f"Error normalizing gene data: {e}")
        normalized_gene_data = gene_data  # Use original data if normalization fails
else:
    print("No gene expression data found.")
    normalized_gene_data = pd.DataFrame()

# 2. Link clinical and genetic data
if is_gene_available and is_trait_available:
    # Ensure samples in both dataframes match by getting common sample IDs
    gene_samples = set(normalized_gene_data.columns)
    clinical_samples = set(clinical_df_t.index)
    common_samples = list(gene_samples.intersection(clinical_samples))
    
    # Print sample ID diagnostics
    print(f"Gene data sample IDs (first 5): {list(gene_samples)[:5]}")
    print(f"Clinical data sample IDs (first 5): {list(clinical_samples)[:5]}")
    
    if not common_samples:
        print("No matching samples between gene and clinical data")
        linked_data = pd.DataFrame()
        is_trait_available = False
        is_biased = True  # Set default value for is_biased when no matching samples
        note = f"No matching samples between clinical and gene expression data. Cannot link the datasets."
    else:
        print(f"Found {len(common_samples)} matching samples")
        
        # Subset data to only include common samples
        gene_data_subset = normalized_gene_data[common_samples].T
        clinical_data_subset = clinical_df_t.loc[common_samples]
        
        # Link the data
        linked_data = pd.concat([clinical_data_subset, gene_data_subset], axis=1)
        print(f"Linked data shape: {linked_data.shape}")
        
        # 3. Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # 4. Check for data bias
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        
        note = f"Dataset contains gene expression data and {trait} trait information derived from time_point data (Baseline vs Follow-up)."
else:
    linked_data = pd.DataFrame()
    is_biased = True
    
    if not is_gene_available:
        note = f"Dataset does not contain usable gene expression data."
    elif not is_trait_available:
        note = f"Dataset does not contain {trait} trait information."
    else:
        note = f"Dataset lacks both gene expression and {trait} trait data."

# 5. Final validation
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not usable for {trait} association studies. Not saving linked data.")

Loaded clinical data with shape: (1, 30)
Transposed clinical data shape: (30, 1)
Gene data shape before normalization: (192, 652)


Gene data shape after normalization: (191, 652)


Normalized gene data saved to ../../output/preprocess/Hepatitis/gene_data/GSE85550.csv
Gene data sample IDs (first 5): ['GSM4557563', 'GSM5517540', 'GSM4557443', 'GSM5517446', 'GSM4557547']
Clinical data sample IDs (first 5): ['GSM2279302', 'GSM2279294', 'GSM2279292', 'GSM2279295', 'GSM2279308']
No matching samples between gene and clinical data
Abnormality detected in the cohort: GSE85550. Preprocessing failed.
Dataset is not usable for Hepatitis association studies. Not saving linked data.
